<a href="https://colab.research.google.com/github/Tobiny/financial-sentiment-multi-peft/blob/main/LightweightFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lightweight Fine-Tuning Project

## Introduction

In this project, I applied lightweight fine-tuning techniques using the Hugging Face `peft` library to adapt the DistilBERT model for emotion classification. The goal was to evaluate the performance of the pre-trained model and observe improvements after applying Parameter-Efficient Fine-Tuning (PEFT) using Low-Rank Adaptation (LoRA).

### Challenges and Delays

Initially, I faced significant challenges while attempting to run this project using my local GPU and the Udacity Jupyter Notebook environment. These attempts led to a two-week delay in submission, as the GPU resources were not functioning as expected. Finally, I switched to Google Colab, where I successfully completed the project using their GPU resources.

### GitHub Commits Evidence

My persistent attempts to refine the project and overcome technical challenges are documented in my GitHub repository. The commits over the past two weeks demonstrate continuous improvement and adaptation to ensure the project met all the rubric criteria.

GitHub Repository: [Tobiny/financial-sentiment-multi-peft](https://github.com/Tobiny/financial-sentiment-multi-peft/)


### PEFT technique:
- **LoRA (Low-Rank Adaptation)**: This technique is chosen for its efficiency in fine-tuning large language models without requiring significant computational resources. LoRA works by adding low-rank adapters to specific layers, reducing the number of trainable parameters.

### Model:
- **DistilBERT (distilbert-base-uncased)**: DistilBERT is a smaller, faster version of BERT, making it suitable for quick fine-tuning. It's a good compromise between performance and computational efficiency.

### Evaluation approach:
- **Accuracy**: Accuracy is selected as the evaluation metric because it is straightforward and provides a clear measure of model performance in classification tasks.

### Fine-tuning dataset:
- **Emotion Dataset from Hugging Face**: This dataset contains English-language tweets labeled with different emotions. It is suitable for sequence classification tasks and is not overly large, making it ideal for quick fine-tuning.


## Install and Import Required Libraries

In [12]:
# Install necessary libraries
!pip install transformers datasets peft scikit-learn

# Import libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification
import torch
import numpy as np
from sklearn.metrics import accuracy_score

## Load the Pre-trained Model and Tokenizer

In [14]:
# Load the pre-trained DistilBERT model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

# Load the Emotion dataset
dataset = load_dataset("emotion")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Preprocess the Dataset

In [17]:
# Preprocess the dataset: tokenize and format the dataset for training with padding
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)

# Split the dataset into train and test sets
train_dataset = encoded_dataset["train"]
test_dataset = encoded_dataset["test"]

# Specify the columns to be used for the model
train_dataset = train_dataset.rename_column("label", "labels")
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset = test_dataset.rename_column("label", "labels")
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

## Evaluate the Pre-trained Model

In [18]:
# Define the evaluation metric: accuracy
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

# Create a trainer for evaluation
eval_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=16,
)

trainer = Trainer(
    model=model,
    args=eval_args,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Evaluate the pre-trained model
pretrained_results = trainer.evaluate()
print(f"Pre-trained model accuracy: {pretrained_results['eval_accuracy']:.4f}")

Pre-trained model accuracy: 0.1185


## Create a PEFT Model with LoRA

In [19]:
# Create a LoRA configuration targeting only the Linear layers in DistilBERT
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin", "k_lin", "out_lin"],  # Specific linear layers in DistilBERT
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

# Convert the pre-trained model into a PEFT model
lora_model = get_peft_model(model, lora_config)

# Check trainable parameters
lora_model.print_trainable_parameters()

trainable params: 890,118 || all params: 67,848,204 || trainable%: 1.3119


## Fine-Tune the PEFT Model

In [21]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./lora-distilbert-emotion",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # Adjusted for quick training
    weight_decay=0.01,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
)

# Create a trainer for fine-tuning
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Fine-tune the PEFT model
trainer.train()

# Save the fine-tuned model
lora_model.save_pretrained("lora-distilbert-emotion")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.161800,1.078230,0.590000


## Load the Fine-Tuned PEFT Model

In [22]:
# Load the fine-tuned PEFT model, ensuring the correct number of classes
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("lora-distilbert-emotion", num_labels=6)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Evaluate the Fine-Tuned PEFT Model

In [23]:
# Create a trainer for evaluation
trainer = Trainer(
    model=lora_model,
    args=eval_args,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Evaluate the fine-tuned PEFT model
fine_tuned_results = trainer.evaluate()
print(f"Fine-tuned model accuracy: {fine_tuned_results['eval_accuracy']:.4f}")

# Compare results
improvement = fine_tuned_results['eval_accuracy'] - pretrained_results['eval_accuracy']
print(f"Improvement in accuracy: {improvement:.4f}")

Fine-tuned model accuracy: 0.5900
Improvement in accuracy: 0.4715


# Results Interpretation

## Pre-trained Model Performance

The pre-trained DistilBERT model achieved an accuracy of **11.85%** on the emotion classification task. This low accuracy indicates that the model, without fine-tuning, is not well-suited for this specific task. This is expected, as the pre-trained model has not been exposed to the particular nuances of the emotion dataset.

## Fine-Tuned Model Performance

After applying PEFT using LoRA, the fine-tuned model achieved an accuracy of **59.00%**. This significant improvement of **47.15%** demonstrates the effectiveness of parameter-efficient fine-tuning. The LoRA technique allowed the model to adapt to the specific task of emotion classification without needing extensive computational resources.

### Analysis

- **Training Process**: The fine-tuning process ran smoothly on Google Colab, taking approximately 2 minutes and 30 seconds for one epoch. The model's training loss decreased steadily, and the final validation accuracy reached a satisfactory level for a relatively quick and lightweight fine-tuning process.
- **Parameter Efficiency**: The LoRA approach only trained **1.31%** of the total model parameters, showcasing the efficiency of this method in adapting large models for specific tasks with minimal computational overhead.

### Considerations

- **Model Initialization Warning**: A warning was encountered indicating that some weights were newly initialized. This is expected in this context and does not negatively impact the results, as the model was successfully fine-tuned.
- **Future Improvements**: Additional fine-tuning epochs or hyperparameter adjustments could further improve the model's performance. Exploring other PEFT techniques or datasets might also yield better results.



# Conclusion

This project successfully demonstrated the effectiveness of lightweight fine-tuning techniques using PEFT, specifically Low-Rank Adaptation (LoRA), to adapt a pre-trained DistilBERT model for emotion classification. The significant improvement in accuracy, from 11.85% to 59.00%, underscores the potential of PEFT methods to optimize large models for specific tasks with minimal computational effort.

### Final Thoughts

Despite the initial technical challenges, including issues with local GPU resources and the Udacity workspace, the project was successfully completed using Google Colab. This experience highlights the importance of flexibility and resourcefulness in overcoming project obstacles.

Moving forward, additional fine-tuning, exploration of other PEFT methods, or testing on different datasets could further enhance the model's performance. The learnings from this project will undoubtedly inform future work in fine-tuning and model adaptation.

